# GRU를 이용한 텍스트 생성
- 데이터 다운로드: 캐글 [New York Times Comments](https://www.kaggle.com/aashita/nyt-comments)
- ArticlesApril2018.csv

In [1]:
import numpy as np
import tensorflow as tf

seed = 2021
np.random.seed(seed)
tf.random.set_seed(seed)

In [2]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

### 파일 업로드

In [3]:
from google.colab import files

uploaded = files.upload()
filename = list(uploaded.keys())[0]

Saving ArticlesApril2018.csv to ArticlesApril2018.csv


In [4]:
import pandas as pd

df = pd.read_csv(filename, encoding='latin1')
df.head(2)

,articleID,articleWordCount,byline,documentType,headline,keywords,multimedia,newDesk,printPage,pubDate,sectionName,snippet,source,typeOfMaterial,webURL
0,5adf6684068401528a2aa69b,781,By JOHN BRANCH,article,Former N.F.L. Cheerleadersâ Settlement Offer...,"['Workplace Hazards and Violations', 'Football...",68,Sports,0,2018-04-24 17:16:49,Pro Football,"âI understand that they could meet with us, ...",The New York Times,News,https://www.nytimes.com/2018/04/24/sports/foot...
1,5adf653f068401528a2aa697,656,By LISA FRIEDMAN,article,E.P.A. to Unveil a New Rule. Its Effect: Less ...,"['Environmental Protection Agency', 'Pruitt, S...",68,Climate,0,2018-04-24 17:11:21,Unknown,The agency plans to publish a new regulation T...,The New York Times,News,https://www.nytimes.com/2018/04/24/climate/epa...


### 데이터 전처리

In [5]:
df.columns

Index(['articleID', 'articleWordCount', 'byline', 'documentType', 'headline',
       'keywords', 'multimedia', 'newDesk', 'printPage', 'pubDate',
       'sectionName', 'snippet', 'source', 'typeOfMaterial', 'webURL'],
      dtype='object')

In [6]:
# headline 열의 데이터만 사용, Null 검사
df.headline.isnull().sum()

0

In [7]:
# headline 열의 데이터로 리스트 생성
headline_list = [text for text in df.headline.values]
len(headline_list)

1324

In [8]:
df.shape

(1324, 15)

In [9]:
headline_list[:5]

['Former N.F.L. Cheerleadersâ\x80\x99 Settlement Offer: $1 and a Meeting With Goodell',
 'E.P.A. to Unveil a New Rule. Its Effect: Less Science in Policymaking.',
 'The New Noma, Explained',
 'Unknown',
 'Unknown']

In [10]:
# 노이즈 데이터('Unknown') 제거
headline_list = [text for text in headline_list if text != 'Unknown']
len(headline_list)

1214

In [11]:
# 구두점 제거, 소문자로 변환
from string import punctuation
punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [12]:
def preprocessing(s):
    s = s.encode('utf8').decode('ascii', 'ignore')
    return ''.join(c for c in s if c not in punctuation).lower()

In [13]:
headlines = [preprocessing(x) for x in headline_list]
headlines[:3]

['former nfl cheerleaders settlement offer 1 and a meeting with goodell',
 'epa to unveil a new rule its effect less science in policymaking',
 'the new noma explained']

In [14]:
# 단어 집합을 만들고, 그 크기를 확인
t = Tokenizer()
t.fit_on_texts(headlines)
vocab_size = len(t.word_index) + 1
print(f'단어 집합의 크기: {vocab_size}')

단어 집합의 크기: 3494


In [15]:
sequences = []
for line in headlines:      
    encoded = t.texts_to_sequences([line])[0]
    for i in range(1, len(encoded)):
        sequence = encoded[:i+1]
        sequences.append(sequence)

sequences[:11]

[[99, 269],
 [99, 269, 371],
 [99, 269, 371, 1115],
 [99, 269, 371, 1115, 582],
 [99, 269, 371, 1115, 582, 52],
 [99, 269, 371, 1115, 582, 52, 7],
 [99, 269, 371, 1115, 582, 52, 7, 2],
 [99, 269, 371, 1115, 582, 52, 7, 2, 372],
 [99, 269, 371, 1115, 582, 52, 7, 2, 372, 10],
 [99, 269, 371, 1115, 582, 52, 7, 2, 372, 10, 1116],
 [100, 3]]

In [16]:
index_to_word = {}
for key, value in t.word_index.items():
    index_to_word[value] = key

print('빈도수 상위 1번 단어:', index_to_word[1])
print('빈도수 상위 582번째 단어:', index_to_word[582])

빈도수 상위 1번 단어: the
빈도수 상위 582번째 단어: offer


In [17]:
# 모든 샘플에서 길이가 가장 긴 샘플의 길이 출력
max_len = max(len(s) for s in sequences) 
print('샘플의 최대 길이 :', max_len)

샘플의 최대 길이 : 24


In [18]:
# 전체 샘플의 길이를 24(가장 긴 샘플의 길이)로 패딩
# 'pre' 옵션을 주면 앞을 0으로 패딩
sequences = pad_sequences(sequences, maxlen=max_len, padding='pre')
sequences[:3]

array([[   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          99,  269],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,   99,
         269,  371],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,   99,  269,
         371, 1115]], dtype=int32)

In [19]:
X = sequences[:, :-1]
y = sequences[:, -1]
Y = to_categorical(y, vocab_size)
X.shape, Y.shape

((7803, 23), (7803, 3494))

### 모델 정의/설정/학습
- Embedding
- GRU



In [20]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GRU, Dense

In [21]:
# 임베딩 벡터 10차원, GRU 노드 크기는 128
model = Sequential([ 
    Embedding(vocab_size, 10, input_length=max_len-1),
    GRU(128),
    Dense(vocab_size, activation='softmax')                
])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 23, 10)            34940     
_________________________________________________________________
gru (GRU)                    (None, 128)               53760     
_________________________________________________________________
dense (Dense)                (None, 3494)              450726    
Total params: 539,426
Trainable params: 539,426
Non-trainable params: 0
_________________________________________________________________


In [22]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [23]:
history = model.fit(X, Y, epochs=200, verbose=2)

Epoch 1/200
244/244 - 34s - loss: 7.7057 - accuracy: 0.0291
Epoch 2/200
244/244 - 1s - loss: 7.1229 - accuracy: 0.0315
Epoch 3/200
244/244 - 1s - loss: 6.9182 - accuracy: 0.0432
Epoch 4/200
244/244 - 1s - loss: 6.7109 - accuracy: 0.0495
Epoch 5/200
244/244 - 1s - loss: 6.4869 - accuracy: 0.0534
Epoch 6/200
244/244 - 1s - loss: 6.2371 - accuracy: 0.0556
Epoch 7/200
244/244 - 1s - loss: 5.9657 - accuracy: 0.0624
Epoch 8/200
244/244 - 1s - loss: 5.6906 - accuracy: 0.0679
Epoch 9/200
244/244 - 1s - loss: 5.4227 - accuracy: 0.0759
Epoch 10/200
244/244 - 1s - loss: 5.1653 - accuracy: 0.0834
Epoch 11/200
244/244 - 1s - loss: 4.9260 - accuracy: 0.0951
Epoch 12/200
244/244 - 1s - loss: 4.6989 - accuracy: 0.1153
Epoch 13/200
244/244 - 1s - loss: 4.4823 - accuracy: 0.1402
Epoch 14/200
244/244 - 1s - loss: 4.2784 - accuracy: 0.1654
Epoch 15/200
244/244 - 1s - loss: 4.0816 - accuracy: 0.2011
Epoch 16/200
244/244 - 1s - loss: 3.8975 - accuracy: 0.2302
Epoch 17/200
244/244 - 1s - loss: 3.7178 - accur

### 모델 검증

In [24]:
# 모델 검증용 문장을 생성하는 함수
def sentence_generation(model, t, current_word, n): # 모델, 토크나이저, 현재 단어, 반복할 횟수
    init_word = current_word # 처음 들어온 단어도 마지막에 같이 출력하기위해 저장
    sentence = ''
    for _ in range(n): # n번 반복
        encoded = t.texts_to_sequences([current_word])[0] # 현재 단어에 대한 정수 인코딩
        encoded = pad_sequences([encoded], maxlen=23, padding='pre') # 데이터에 대한 패딩
        #result = model.predict_classes(encoded, verbose=0)
        result = np.argmax(model.predict(encoded), axis=-1)
    # 입력한 X(현재 단어)에 대해서 Y를 예측하고 Y(예측한 단어)를 result에 저장.
        for word, index in t.word_index.items(): 
            if index == result: # 만약 예측한 단어와 인덱스와 동일한 단어가 있다면
                break # 해당 단어가 예측 단어이므로 break
        current_word = current_word + ' ' + word # 현재 단어 + ' ' + 예측 단어를 현재 단어로 변경
        sentence = sentence + ' ' + word # 예측 단어를 문장에 저장
        
    # for문이므로 이 행동을 다시 반복
    sentence = init_word + sentence
    return sentence

In [25]:
print(sentence_generation(model, t, 'i', 10))
print(sentence_generation(model, t, 'epa', 10))
print(sentence_generation(model, t, 'former', 10))

i want to be rich and im not sorry scientists our
epa sheriff indulged pruitt as security spending mounted on his viral
former nfl cheerleaders settlement offer 1 and a meeting with goodell
